# Лабораторная работа 1. Кластерный анализ

In [205]:
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline

from sklearn import preprocessing

import os

In [2]:
from sklearn.cluster import DBSCAN

In [206]:
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster

In [225]:
data = pd.read_csv('Econom_Cities_data.csv', sep = ';',index_col='City',decimal=',')
data.head()

,Work,Price,Salary
City,,,
Amsterdam,1714,65.6,49.0
Athens,1792,53.8,30.4
Bogota,2152,37.9,11.5
Bombay,2052,30.3,5.3
Brussels,1708,73.8,50.5


In [228]:
data_2 = data.copy()  #создаём копию для иерархического анализа

# 3.DBScan

In [226]:
# оставляем выбросы - dbscan должен их распознать
# стандартизуем данные
stand_1 = preprocessing.StandardScaler()

# параметры стандартизации
stand_1.fit(data)
# преобразование
X = stand_1.transform(data)
X = pd.DataFrame(X, index=data.index, columns=data.columns)

X.head()

,Work,Price,Salary
City,,,
Amsterdam,0.138269,-0.151250,0.213212
Athens,0.171046,-0.698647,0.203941
Bogota,0.322325,-1.436242,0.194519
Bombay,0.280303,-1.788804,0.191429
Brussels,0.135748,0.229145,0.213960


In [105]:
# создаём 1-ую модель с параметрами по умолчанию
dbscan_1 = DBSCAN(eps=0.5, metric='euclidean', min_samples=5)

dbscan_1.fit(X)

dbscan_1.labels_

array([ 0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,
       -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,
        0,  0,  0,  0,  0,  0, -1,  0,  0,  0, -1,  0,  0,  0],
      dtype=int64)

Есть несколько выбросов, но большинство наблюдений попало в единственный кластер. Значит, радиус слишком большой.

In [138]:
dbscan_2 = DBSCAN(eps=0.25, metric='euclidean', min_samples=5)

dbscan_2.fit(X)

dbscan_2.labels_

array([ 0,  0, -1, -1,  0,  0, -1,  0,  0, -1,  0,  0,  0, -1, -1,  0,  0,
       -1,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0, -1,
        0,  0,  0,  0,  0,  0, -1,  0,  0,  0, -1,  0,  0, -1],
      dtype=int64)

После уменьшения радиуса в 2 раза ситуация примерно та же - продолжаем уменьшать радиус.

In [139]:
dbscan_3 = DBSCAN(eps=0.2, metric='euclidean', min_samples=5)

dbscan_3.fit(X)

dbscan_3.labels_

array([ 2,  0, -1, -1,  1,  0, -1,  2,  1, -1,  1,  1,  1, -1, -1,  2,  1,
       -1,  0,  0,  0,  0,  1,  1,  1, -1, -1,  0,  1,  1,  0,  1,  0, -1,
        0,  1,  0,  0, -1,  2, -1,  1,  1,  1, -1,  1,  1, -1],
      dtype=int64)

Образовалось несколько кластеров, однако  резко возросло число выбросов. Делаем вывод, что m также нужно уменьшить.

In [145]:
dbscan_4 = DBSCAN(eps=0.2, metric='euclidean', min_samples=3)

dbscan_4.fit(X)

dbscan_4.labels_

array([ 0,  0,  0, -1,  0,  0, -1,  0,  0,  1,  0,  0,  0,  1,  2,  0,  0,
       -1,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  2,
        0,  0,  0,  0,  0,  0,  2,  0,  0,  0,  2,  0,  0,  1],
      dtype=int64)

In [150]:
data['dbscan_4'] = dbscan_4.labels_
data[data['dbscan_4']==0].index.values

array(['Amsterdam', 'Athens', 'Bogota', 'Brussels', 'Buenos_Aires',
       'Caracas', 'Chicago', 'Dublin', 'Dusseldorf', 'Frankfurt',
       'Hong_Kong', 'Houston', 'Johannesburg', 'Kuala_Lumpur', 'Lagos',
       'Lisbon', 'London', 'Los_Angeles', 'Luxembourg', 'Manila',
       'Mexico_City', 'Milan', 'Montreal', 'Nairobi', 'New_York',
       'Nicosia', 'Panama', 'Paris', 'Rio_de_Janeiro', 'San_Paulo',
       'Seoul', 'Singpore', 'Sydney', 'Taipei', 'Tel_Aviv', 'Toronto',
       'Vienna'], dtype=object)

Выбросов стало меньше, но состав 0-го кластера нелогичен: в него входят как развитые города Европы и Америки, так и бедные города Африки и Азии. Вероятно, радиус всё ещё достаточно велик - снижаем его небольшими шагами.

In [233]:
dbscan_5 = DBSCAN(eps=0.15, metric='euclidean', min_samples=3)

dbscan_5.fit(X)

dbscan_5.labels_

array([ 0,  1, -1, -1,  0,  1, -1, -1,  0,  3,  0,  0,  0,  3,  2,  0,  0,
       -1,  1,  1,  1,  1,  0,  0,  0,  3, -1,  1,  0,  0,  1,  0,  1,  2,
        1,  0,  1,  1,  1,  0,  2,  0,  0,  0,  2,  0,  0, -1],
      dtype=int64)

In [235]:
data['dbscan_5'] = dbscan_5.labels_
data.groupby('dbscan_5').mean()

,Work,Price,Salary
dbscan_5,,,
-1,-1373.857143,49.985714,-2838.042857
0,1866.571429,74.652381,49.547619
1,1896.538462,50.100000,15.861538
2,1733.750000,113.850000,59.375000
3,1769.000000,93.666667,67.733333


In [175]:
dbscan_6 = DBSCAN(eps=0.14, metric='euclidean', min_samples=3)

dbscan_6.fit(X)

dbscan_6.labels_

array([-1,  0, -1, -1,  1,  0, -1, -1,  1,  5,  1,  1,  1,  5,  2, -1,  1,
       -1,  3,  3,  3,  0,  4, -1,  1,  5, -1,  3,  4,  1,  3,  4,  3,  2,
        3,  4,  3,  3,  0, -1,  2,  1,  4, -1,  2,  1,  1, -1],
      dtype=int64)

Больше уменьшать радиус нет смысла - снова возрастает число выбросов и появляются кластеры минимального размера

In [241]:
data[data['dbscan_5']==-1].index.values  # выбросы

array(['Bogota', 'Bombay', 'Cairo', 'Caracas', 'Jakarta', 'Manila',
       'Zurich'], dtype=object)

In [237]:
data[data['dbscan_5']==0].index.values

array(['Amsterdam', 'Brussels', 'Chicago', 'Dublin', 'Dusseldorf',
       'Frankfurt', 'Hong_Kong', 'Houston', 'London', 'Los_Angeles',
       'Luxembourg', 'Milan', 'Montreal', 'New_York', 'Paris', 'Singpore',
       'Sydney', 'Taipei', 'Tel_Aviv', 'Toronto', 'Vienna'], dtype=object)

In [238]:
data[data['dbscan_5']==1].index.values

array(['Athens', 'Buenos_Aires', 'Johannesburg', 'Kuala_Lumpur', 'Lagos',
       'Lisbon', 'Mexico_City', 'Nairobi', 'Nicosia', 'Panama',
       'Rio_de_Janeiro', 'San_Paulo', 'Seoul'], dtype=object)

In [239]:
data[data['dbscan_5']==2].index.values

array(['Helsinki', 'Oslo', 'Stockholm', 'Tokyo'], dtype=object)

In [240]:
data[data['dbscan_5']==3].index.values

array(['Copenhagen', 'Geneva', 'Madrid'], dtype=object)

# Сравнение с иерархическим анализом

In [229]:
# очищаем датасет от выбросов
data_out = data_2.drop('Cairo')
data_out = data_out.drop('Jakarta')
data_out.head()

,Work,Price,Salary
City,,,
Amsterdam,1714,65.6,49.0
Athens,1792,53.8,30.4
Bogota,2152,37.9,11.5
Bombay,2052,30.3,5.3
Brussels,1708,73.8,50.5


In [230]:
# повторяем иерархический анализ
stand_2 = preprocessing.StandardScaler()

# параметры стандартизации
stand_2.fit(data_out)
# преобразование
X_2 = stand_2.transform(data_out)
X_2 = pd.DataFrame(X_2, index=data_out.index, columns=data_out.columns)

X_2.head()

,Work,Price,Salary
City,,,
Amsterdam,-0.962166,-0.212712,0.386095
Athens,-0.509827,-0.770489,-0.373488
Bogota,1.577891,-1.522069,-1.145323
Bombay,0.997969,-1.881316,-1.398517
Brussels,-0.996961,0.174896,0.447351


In [231]:
link_2 = linkage(X_2, 'ward', 'euclidean')

data_out['cluster_2'] = fcluster(link_2, 4, criterion='distance')

In [236]:
# таблица сопряжённости
tab2 = pd.crosstab(data['dbscan_5'], data_out['cluster_2'])

print(tab2)

cluster_2  1   2  3  4
dbscan_5              
-1         1   0  0  4
 0         0  17  0  4
 1         0   0  7  6
 2         4   0  0  0
 3         1   2  0  0


Наиболее неизменённым остаётся кластер развитых городов (2-ой в kmeans, 0-ой в dbscan), другие кластеры либо бьются на более мелкие, либо объединяются.

# Выводы

В целом каждый метод кластеризации на выходе даёт "шкалу" стран по уровню развития, но акценты расставлены несколько по-разному.

1. Иерархический анализ

Даёт самый логичный и прозрачный результат: Передовые города - Развитые города - Развивающиеся города с развитой сферой услуг (туризм) - Развивающиеся города с преимущественно традиционным хозяйством

2. Kmeans

Развивающиеся города объединяются в один кластер. Добавляется новый кластер развитых транспортных узлов с характерной экономикой.

3. DBScan 

Наибеднейшие города идентифицируются как выбросы, развивающиеся города объединяются в один кластер. В выбросы попадает и самый дорогой в выборке Цюрих, а передовые города бьются на два небольших кластера (разница в соотношении уровня цен и зп).